## analyse the study and results

In [1]:
import pickle as pkl
from typing import Any, Dict
import os
import sys
import pandas as pd
import numpy as np
import seaborn as sns
import math
import json
import datetime
import matplotlib.pyplot as plt 
%matplotlib inline
sys.path.append(os.path.abspath('./..'))
# from src.env_custom import CartPoleButter
import gym
import torch
import torch.nn as nn
from stable_baselines3 import DQN
from stable_baselines3.common.callbacks import EvalCallback

import optuna
from optuna.pruners import MedianPruner
from optuna.samplers import TPESampler
from optuna.visualization import plot_optimization_history, plot_param_importances

with open("./study.pkl", "rb") as f:
# with open("./old_study/study.pkl", "rb") as f:
    study = pkl.load(f)

fig1 = plot_optimization_history(study)
fig2 = plot_param_importances(study)
fig2.write_image('params.pdf')
fig1.write_image('hist.pdf')
fig1.show()
fig2.show()

OSError: libmpi_cxx.so.20: cannot open shared object file: No such file or directory

In [ ]:
df=study.trials_dataframe()
df.head()


In [ ]:
df.sort_values(by=['value'],ascending=False)

In [ ]:
print(df.loc[df['value'].argmax()])